In [1]:
%load_ext autoreload
%autoreload 2
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import Vantage
from pylabrobot.resources.hamilton import VantageDeck
backend = VantageDeck(size=1.3)
lh = LiquidHandler(backend=Vantage(), deck=backend)

In [ ]:
await lh.setup()


In [ ]:
import sys; print(sys.path)

In [ ]:
import logging
from pathlib import Path
from pylabrobot.config import Config

config = Config(
  logging=Config.Logging(
    level=logging.DEBUG,
    log_dir=Path("my_logs")
  )
)

In [2]:
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cos_96_DW_1mL,
    PCR_Plate_96_Well,
    Cos_96_PCR,
    Cos_96_DW_2mL,
    HTF_L,  #""" Tip Rack with 96 1000ul High Volume Tip with filter """
    LT_L,   #""" Tip Rack with 96 10ul Low Volume Tip with filter """
    STF_L,  #""" Tip Rack with 96 300ul Standard Volume Tip with filter """
)

ImportError: cannot import name 'Cos_96_DW_1mL' from 'pylabrobot.resources' (/Users/danielkozyrytski/pylabrobot/pylabrobot/resources/__init__.py)

In [ ]:
lh.summary()

In [ ]:
tip_car = TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = LT_L(name='p_10')
tip_car[1] = STF_L(name='p_300')
tip_car[2] = HTF_L(name='p_1000')



plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = Cos_96_DW_1mL(name='final')
plt_car[1] = Cos_96_PCR(name='source')
plt_car[2] = Cos_96_DW_2mL(name='buffer')
 

In [ ]:
lh.deck.assign_child_resource(tip_car, rails=24)
lh.deck.assign_child_resource(plt_car, rails=17)
lh.summary()

In [ ]:
p_10 = lh.deck.get_resource("p_10")
p_300 = lh.deck.get_resource("p_300")
p_1000 = lh.deck.get_resource("p_1000")

q_bitp = lh.deck.get_resource("final")
buffer = lh.deck.get_resource("buffer")
source = lh.deck.get_resource("source")


In [ ]:
# await lh.pick_up_tips(p_300["A1:H1"])

In [ ]:
# await lh.drop_tips(p_300["A1:H1"])

In [ ]:
print("How many columns do you have?")
#column = int(input())

#column_number = int(input())

a = 199.0
q_bit_volume = [a, a, a, a, a, a, a, a]
column_number = 3 #number of columns that you are using

await lh.pick_up_tips(p_300["A1:H1"])

for i in range(0, column_number):
    await lh.aspirate(buffer["A1:H1"], vols = q_bit_volume)
    await lh.dispense(q_bitp[f"A{i+1}:H{i+1}"], vols = q_bit_volume)
    
#await lh.discard_tips()
await lh.drop_tips(p_300["A1:H1"])



In [ ]:
#THE NEW STRATEGY OF GETTING 1 UL

column_number = 3 #number of columns that you are using

mix_cycles = 3 #number of mix cycles after dispencing dna into the q_bit buffer 

dna_qbit_vol = 1.0
b = 10.0 #mix volume
c = 5 #number of mixing cycles
mix_v = [b, b, b, b, b, b, b, b]
dna_volume = [dna_qbit_vol, dna_qbit_vol, dna_qbit_vol, dna_qbit_vol, dna_qbit_vol, dna_qbit_vol, dna_qbit_vol, dna_qbit_vol]

for j in range(0, column_number):
    await lh.pick_up_tips(p_10[f"A{j+1}:H{j+1}"])
    await lh.aspirate(source[f"A{j+1}:H{j+1}"], vols = [10]*8)
    await lh.dispense(source[f"A{j+1}:H{j+1}"], vols = [9]*8)
    await lh.aspirate(q_bitp[f"A{j+1}:H{j+1}"], vols = [7]*8)
    await lh.dispense(q_bitp[f"A{j+1}:H{j+1}"], vols = [8]*8)
    for g in range (0, mix_cycles):
            await lh.aspirate(q_bitp[f"A{j+1}:H{j+1}"], vols = mix_v)
            await lh.dispense(q_bitp[f"A{j+1}:H{j+1}"], vols = mix_v) 

    await lh.discard_tips()
    # await lh.drop_tips(p_10[f"A{j+1}:H{j+1}"])


In [ ]:
dna_qbit_vol = 1.0
column_number = 1 #number of columns that you are using
b = 10.0 #mix volume
c = 5 #number of mixing cycles
mix_v = [b, b, b, b, b, b, b, b]
mix_c = 5
dna_volume = [dna_qbit_vol, dna_qbit_vol, dna_qbit_vol, dna_qbit_vol, dna_qbit_vol, dna_qbit_vol, dna_qbit_vol, dna_qbit_vol]
for j in range(0, column_number):
    await lh.pick_up_tips(p_10[f"A{j+1}:H{j+1}"])
    await lh.aspirate(source[f"A{j+1}:H{j+1}"], vols = dna_volume)
    await lh.dispense(q_bitp[f"A{j+1}:H{j+1}"], vols = dna_volume)
#     await lh.dispense(q_bitp[f"A{j+1}:H{j+1}"], vols = dna_volume, mix_volume = mix_v, mix_cycles = mix_c)
#     await lh.dispense(q_bitp[f"A{j+1}:H{j+1}"], vols = dna_volume, mix_volume = [10.0]*8, mix_cycles = 5)
#, mix_volume = 10.0, mix_cycles = 5
    for g in range (0, 3):
            await lh.aspirate(q_bitp[f"A{j+1}:H{j+1}"], vols = mix_v)
            await lh.dispense(q_bitp[f"A{j+1}:H{j+1}"], vols = mix_v) 

    await lh.discard_tips()
    #await lh.drop_tips(p_10[f"A{j+1}:H{j+1}"])


Put the plate in the plate reader and export the concentrations as .csv

In [ ]:
await lh.drop_tips(p_10["A3:H3"])

In [ ]:
await lh.pick_up_tips(p_10["A6:H6"])


In [ ]:
await lh.discard_tips()

In [ ]:
# #await lh.aspirate(buffer["A1:H1"], vols = 199.0)
# a = 199.0
# #q_bit_volume = [199.0, 199.0, 199.0, 199.0, 199.0, 199.0, 199.0, 199.0]
# q_bit_volume = [a, a, a, a, a, a, a, a]
# await lh.dispense(buffer["A1:H1"], vols = q_bit_volume)

In [ ]:
await lh.discard_tips()

In [ ]:
await lh.drop_tips(tips_300["A1:H1"])

In [ ]:
await lh.pick_up_tips(tips_10["A1:H1"])

In [ ]:
for i in range (1, column):
    await lh.aspirate(source[f"A{i}:H{i}"], vols = 1.0)
    await lh.drop_tips(trash)
    await lh.dispense(q_bitp[f"A{i}:H{i}"], vols = 1.0, mix_volume = 10.0, mix_cycles = 10)